In [1]:
import json
import requests
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 12]

In [2]:
import networkx as nx

from networkx.readwrite import json_graph
from networkx.algorithms import community